In [1]:
import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras import backend as K

C:\Users\Public\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
X_train = np.load('X-train.npy')
y_train = np.load('y-train.npy')
X_test = np.load('X-test.npy')
y_test = np.load('y-test.npy')

In [3]:
X_train

array([[[ 99,  85,  78, ...,  73,  59,  52],
        [ 93,  86,  82, ...,  65,  48,  42],
        [ 83,  88,  86, ...,  64,  42,  32],
        ...,
        [ 52,  49,  59, ..., 131, 136, 127],
        [ 74,  76,  82, ..., 143, 148, 139],
        [ 79,  92, 102, ..., 124, 133, 127]],

       [[  0,   0,   1, ...,   0,   0,   0],
        [  0,   0,   1, ...,   0,   0,   0],
        [  0,   0,   1, ...,   0,   0,   0],
        ...,
        [  2,   0,   0, ...,   0,   0,   0],
        [  2,   1,   0, ...,   0,   0,   0],
        [  2,   1,   0, ...,   0,   0,   0]],

       [[240, 240, 237, ..., 139, 134, 131],
        [239, 239, 236, ..., 134, 130, 127],
        [237, 237, 234, ..., 129, 125, 123],
        ...,
        [195, 194, 199, ..., 109, 107, 106],
        [196, 195, 198, ..., 109, 107, 106],
        [197, 196, 198, ..., 109, 107, 106]],

       ...,

       [[194, 195, 195, ..., 190, 192, 194],
        [195, 195, 196, ..., 191, 193, 195],
        [196, 196, 197, ..., 192, 195, 197

In [4]:
X_train = X_train.reshape(4161, 784)
X_test = X_test.reshape(1041, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [5]:
# Convert class vectors to binary class matrices
# So instead of one column with 10 values, create 10 binary columns
y_train = keras.utils.to_categorical(y_train, 3)
y_test = keras.utils.to_categorical(y_test, 3)

In [6]:
# Start with a simple sequential model
model = Sequential()

# Add dense layers to create a fully connected MLP
# Note that we specify an input shape for the first layer, but only the first layer.
# Relu is the activation function used
model.add(Dense(64, activation='relu', input_shape=(784,)))
# Dropout layers remove features and fight overfitting
model.add(Dropout(0.1))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.1))
# End with a number of units equal to the number of classes we have for our outcome
model.add(Dense(3, activation='softmax'))

model.summary()

# Compile the model to put it all together.
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 64)                50240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                4160      
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 195       
Total params: 54,595
Trainable params: 54,595
Non-trainable params: 0
_________________________________________________________________


In [7]:
history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=20,
                    verbose=1,
                    validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 4161 samples, validate on 1041 samples
Epoch 1/20
4161/4161 [==============================] - 1s 142us/step - loss: 0.8712 - acc: 0.6686 - val_loss: 0.7890 - val_acc: 0.7022
Epoch 2/20
4161/4161 [==============================] - 0s 42us/step - loss: 0.7926 - acc: 0.7054 - val_loss: 0.7465 - val_acc: 0.7195
Epoch 3/20
4161/4161 [==============================] - 0s 40us/step - loss: 0.7637 - acc: 0.7099 - val_loss: 0.7473 - val_acc: 0.7464
Epoch 4/20
4161/4161 [==============================] - 0s 39us/step - loss: 0.7219 - acc: 0.7313 - val_loss: 0.7552 - val_acc: 0.7426
Epoch 5/20
4161/4161 [==============================] - 0s 40us/step - loss: 0.7058 - acc: 0.7304 - val_loss: 0.7440 - val_acc: 0.7080
Epoch 6/20
4161/4161 [==============================] - 0s 41us/step - loss: 0.6720 - acc: 0.7395 - val_loss: 0.6202 - val_acc: 0.7646
Epoch 7/20
4161/4161 [==============================] - 0s 41us/step - loss: 0.6576 - acc: 0.7419 - val_loss: 0.6476 - val_acc: 0.7560
Epoch 

In [8]:
X_train = np.load('X-train.npy')
y_train = np.load('y-train.npy')
X_test = np.load('X-test.npy')
y_test = np.load('y-test.npy')

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [10]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 3


if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

X_train shape: (4161, 28, 28, 1)
4161 train samples
1041 test samples
Train on 4161 samples, validate on 1041 samples
Epoch 1/10
4161/4161 [==============================] - 19s 5ms/step - loss: 0.9635 - acc: 0.6114 - val_loss: 0.9303 - val_acc: 0.6196
Epoch 2/10
4161/4161 [==============================] - 18s 4ms/step - loss: 0.9372 - acc: 0.6200 - val_loss: 0.9295 - val_acc: 0.6196
Epoch 3/10
4161/4161 [==============================] - 18s 4ms/step - loss: 0.9404 - acc: 0.6200 - val_loss: 0.9472 - val_acc: 0.6196
Epoch 4/10
4161/4161 [==============================] - 19s 4ms/step - loss: 0.9335 - acc: 0.6200 - val_loss: 0.9348 - val_acc: 0.6196
Epoch 5/10
4161/4161 [==============================] - 18s 4ms/step - loss: 0.9383 - acc: 0.6200 - val_loss: 0.9266 - val_acc: 0.6196
Epoch 6/10
4161/4161 [==============================] - 18s 4ms/step - loss: 0.9335 - acc: 0.6200 - val_loss: 0.9260 - val_acc: 0.6196
Epoch 7/10
4161/4161 [==============================] - 18s 4ms/step - l

In [11]:
X_train = np.load('X-train-rgb.npy')
y_train = np.load('y-train-rgb.npy')
X_test = np.load('X-test-rgb.npy')
y_test = np.load('y-test-rgb.npy')

In [12]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= 255
X_test /= 255

In [13]:
# input image dimensions, from our data
img_rows, img_cols = 28, 28
num_classes = 3


if K.image_data_format() == 'channels_first':
    X_train = X_train.reshape(X_train.shape[0], 3, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 3, img_rows, img_cols)
    input_shape = (3, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 3)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 3)
    input_shape = (img_rows, img_cols, 3)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


# Building the Model
model = Sequential()
# First convolutional layer, note the specification of shape
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train,
          batch_size=128,
          epochs=20,
          verbose=1,
          validation_data=(X_test, y_test))
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

X_train shape: (2381, 28, 28, 3)
2381 train samples
596 test samples
Train on 2381 samples, validate on 596 samples
Epoch 1/20
2381/2381 [==============================] - 12s 5ms/step - loss: 1.0987 - acc: 0.3343 - val_loss: 1.0971 - val_acc: 0.4748
Epoch 2/20
2381/2381 [==============================] - 11s 5ms/step - loss: 1.0991 - acc: 0.3310 - val_loss: 1.0978 - val_acc: 0.3221
Epoch 3/20
2381/2381 [==============================] - 11s 5ms/step - loss: 1.0989 - acc: 0.3268 - val_loss: 1.0980 - val_acc: 0.3221
Epoch 4/20
2381/2381 [==============================] - 12s 5ms/step - loss: 1.0987 - acc: 0.3394 - val_loss: 1.0979 - val_acc: 0.3221
Epoch 5/20
2381/2381 [==============================] - 12s 5ms/step - loss: 1.0985 - acc: 0.3452 - val_loss: 1.0977 - val_acc: 0.3221
Epoch 6/20
2381/2381 [==============================] - 11s 5ms/step - loss: 1.0985 - acc: 0.3381 - val_loss: 1.0977 - val_acc: 0.3221
Epoch 7/20
2381/2381 [==============================] - 12s 5ms/step - los